In [73]:
from IPython.core.display import HTML

with open('style.html', 'r') as file:
    css = file.read()
HTML(css)

# Auswerten eines Schach-Endspiels

Neben der Berechnung der Endspiel-Situationen, gilt es auch zu überprüfen, ob es sich bei den Zügen der KI auch um die besten Züge handelt. Hierfür werden im Rahmen von diesem Notebook Testszenarien geschrieben, die die Entscheidungen der KI bewerten. Zum Vergleich wurde die Engine Stockfish herangezogen. Dabei handelt es sich um die momentan beste Schach-Engine, die online frei zur Verfügung steht. 

Insgesamt können vier unterschiedliche Szenarien in dem Notebook getestet werden. Dazu muss jedoch bereits eine Berechnung mit dem Notebook ``calculation.ipynb`` stattgefunden haben, da anhand dieser berechneten Mengen die Tests durchgeführt werden. Die Struktur der Tests wird wie folgt aufgebaut sein:
* ``compare_fen_stockfish``: Dieser Test überprüft für eine mitgegebene FEN, wie viele Züge die KI und wie viele Züge Stockfish zum Gewinnen braucht. 
* ``test_random_boards``: Dieser Test überprüft für eine angegebene Anzahl n zufälliger Boards, welche der beiden Lösungswege bessere Ergebnisse liefert.
* ``compare_sequence_stockfish``: Dieser Test überprüft für ein mitgegebenes n alle Boards, die in der Menge $S_n$ stehen, ob Stockfish oder die KI bessere Ergebnisse erzielt.
* ``compare_all_sequences``: Dieser Test überprüft für jegliches Board, das in den $S_n$ Mengen zu finden ist, ob Stockfish oder die KI eine bessere Lösung bestimmen kann.

Die Ergebnisse der letzten drei Tests werden in dem Ordner ``/Tests`` gespeichert.

## Importe

Für die Durchführung der Tests und der Auswertung eines Schach-Endspiels gegen Stockfish werden die hier stehenden Bibliotheken benötigt:

* ``chess``: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* ``stockfish``: Bibliothek, welche eine Python Schnittstelle für die Stockfish-Enginge anbietet.
* ``clear_output`` aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen verwendet wird.
* ``display`` aus der IPython-display-Library: Wird zum grafischen Anzeigen der Schachbretter verwendet.
* ``json``: Wird zum Speichern der Ergebnisse verwendet.
* ``time``: Wird zum Verzögern von Ausgaben verwendet.
* ``datetime``: Zur Bestimmung des aktuellen Datums, sodass der Vergleich gespeichert werden kann.

In [74]:
import chess
import stockfish
from IPython.display import clear_output
from IPython.display import display
import time
from datetime import datetime
import pickle

## Konfiguration

Zu Beginn müssen für den Vergleich unterschiedliche Variablen definiert werden. Diese beinhalten eine FEN, die für ein gegebenes Szenario überprüft werden soll, den Pfad zu der berechneten $S_n$ Sequenz und einen Pfad zu der Stockfish-Installation. Weiterhin wird eine globale Variable ``VERBOSE`` festgelegt, die ein Anzeigen von zusätzlichen Kommentaren ermöglicht.

In [75]:
FEN = "4k3/8/8/8/8/8/8/R3K3 w - - 0 1"
STOCKFISH_PATH = "./stockfish/stockfish.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

Der Vergleich der Spielbretter findet in diesem Notebook anhand der FEN-Schreibweise statt. Diese wird in zweierlei Form verwendet, in ihrer gesamten und in einer gekürzten Schreibweise. Die gekürzte Schreibweise beinhaltet nur das momentane Spielfeld, aber auch die Farbe, die gerade am Zug ist. Zur Erstellung dieser Schreibweise wurde die Funktion ``get_board_and_turn`` geschrieben. Diese erhält als Parameter eine komplette ``fen`` und gibt in der FEN-Notation das Brett und die Farbe, die am Zug ist, als String zurück.

In [76]:
def get_board_and_turn(fen):
    split = fen.split()
    short_fen = str(split[0]) + " " + str(split[1])
    return short_fen

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die mit Hilfe des Notebooks ``calculation.ipynb`` bestimmt werden. Diese wurden bereits in einer JSON-Datei serialisiert abgespeichert. Für dieses Notebook werden alle FENs gelesen und daraus Board-Objekte erstellt. Dies geschieht mit der Funktion ``load_s_n_sequence``. Diese erhält den Dateipfad (``filepath``) als Parameter, um aus der Datei die $S_n$ Mengen auslesen zu können. Als Rückgabewert gibt sie sowohl die ausgeschriebenen FENs, als auch die gekürzten FENs zurück.


In [77]:
def load_s_n_fens(filename):
    s_n_sequence_fen = []
    s_n_sequence_short_fen = []
    with (open("S_n_Results/" + filename + ".chessTest", "rb")) as test_fens:
        tmp = pickle.loads(test_fens.read())
        for item in tmp:
            tmp_set_fen = set()
            tmp_set_short = set()
            for fen in item:
                tmp_set_fen.add(fen)
                tmp_set_short.add(get_board_and_turn(fen))
            s_n_sequence_fen.append(tmp_set_fen)
            s_n_sequence_short_fen.append(tmp_set_short)
    return s_n_sequence_fen, s_n_sequence_short_fen

In [92]:
S_N_FILE = "S_n_seq_queen"

S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_fens(S_N_FILE)
print(S_N_Sequence_fen_short[0:2])

[{'3k4/3Q4/2K5/8/8/8/8/8 b', '8/8/8/8/8/5K2/8/5k1Q b', '8/8/8/8/8/7K/8/4Q1k1 b', '8/8/8/8/8/1K6/8/k4Q2 b', '8/8/8/8/8/7K/6Q1/6k1 b', '1Q5k/8/7K/8/8/8/8/8 b', '8/6Qk/5K2/8/8/8/8/8 b', '8/8/5KQk/8/8/8/8/8 b', '8/8/8/8/8/1K6/8/k2Q4 b', '1k1Q4/8/1K6/8/8/8/8/8 b', '8/8/8/8/8/K7/8/k2Q4 b', '8/8/8/Q7/8/8/8/k1K5 b', '8/8/8/8/8/2K5/8/3k1Q2 b', '8/8/8/5KQk/8/8/8/8 b', 'Q7/8/8/8/8/8/8/k1K5 b', '3k3Q/8/3K4/8/8/8/8/8 b', '8/8/8/7Q/8/8/5K1k/8 b', '8/8/8/8/8/6K1/8/2Q4k b', '8/8/8/8/8/6K1/8/5Q1k b', '2Q1k3/8/5K2/8/8/8/8/8 b', '7k/5K2/7Q/8/8/8/8/8 b', '8/5K2/6Qk/8/8/8/8/8 b', 'Q3k3/8/4K3/8/8/8/8/8 b', '5K2/7k/8/7Q/8/8/8/8 b', '8/8/8/2K5/k7/8/Q7/8 b', '5k2/5Q2/5K2/8/8/8/8/8 b', '8/2K5/kQ6/8/8/8/8/8 b', 'k1Q5/8/1K6/8/8/8/8/8 b', '8/8/8/8/8/7K/8/2Q4k b', '8/8/kQK5/8/8/8/8/8 b', '8/8/6Qk/5K2/8/8/8/8 b', '7k/5KQ1/8/8/8/8/8/8 b', '3k1Q2/8/3K4/8/8/8/8/8 b', '8/8/Q7/8/8/8/8/k1K5 b', '8/8/8/7Q/8/8/8/5K1k b', '5K2/6Qk/8/8/8/8/8/8 b', 'Q7/8/8/8/k1K5/8/8/8 b', 'k1K5/8/8/8/8/Q7/8/8 b', '5K1k/8/8/8/8/8/7Q/8 b', '2K5

## Vergleich zwischen Stockfish und KI
Für den Vergleich werden die Anzahlen der Züge zwischen Stockfish und der selbstgeschriebenen KI miteinander verglichen.

### Berechnung der Zuglisten
Der erste Schritt, um eine Zugliste zu erstellen, liegt in der Ermittlung eines Spielzuges für die KI. Hierfür muss zunächst die $S_n$ Menge bestimmt werden, in der die FEN vorliegt. Dies wird mit der Funktion ``find_board_in_sequence`` erreicht. Sie durchsucht eine gegebene $S_n$ Sequenz nach einem übergebenen Schachbrett. Dieses Schachbrett wird in der FEN-Schreibweise übergeben. Zusätzlich wird eine Liste von $S_n$ Mengen übergeben (``sequence_short``), in der letztendlich die ``fen`` gefunden werden soll. Am Ende der Funktion wird der Index der $S_n$ Menge ausgegeben, in der die FEN gefunden wurde. Falls das Board nicht in den übergebenen Mengen gefunden werden kann, wird stattdessen -1 zurückgegeben.

In [79]:
def find_fen_in_sequence(fen, sequence_short):
    short = get_board_and_turn(fen)
    for i in range(len(sequence_short)):
        if short in sequence_short[i]:
            return i
    return -1

Nachdem das Brett in einer $S_n$ Menge gefunden wurde, ist der nächste Schritt den passenden Zug für die KI zu bestimmen. Zu diesem Zweck wurde die Funktion ``find_next_move`` definiert. Diese berechnet für eine übergebene FEN (``fen``) den idealen Spielzug. Weiterhin wird für die Berechnung des Spielzuges der ``s_index`` mitgegen, in dem die FEN gefunden werden kann. Diese wird für die Suche in den "gekürzten", übergebenen $S_n$ Mengen (``s_n_sequence_short``) verwendet. 

Die Funktion liefert als Rückgabewert einerseits den nächsten Move, der für den Test durchgeführt wird. Andererseits wird ein ``s_index`` zurückgegeben. Dieser beschreibt das n der $S_n$ Menge in der die Spielsituation liegt, die nach der Ausführung des Zuges auf dem Schachbrett anzutreffen ist. 

In [80]:
def find_next_move(fen, s_index, s_n_sequence_short):
    STOCKFISH.set_fen_position(fen)
    curr_board = chess.Board(fen)
    if curr_board.turn:
        if VERBOSE:
            print("---White:---")
            print("Starting in S" + str(s_index))
        for move in curr_board.legal_moves:
            curr_board.push(move)
            cur_fen = curr_board.fen()
            _tmp = find_fen_in_sequence(cur_fen, [s_n_sequence_short[s_index - 1]])
            s_index_tmp = s_index - 1
            if _tmp != -1:
                if VERBOSE:
                    print("    Move: " + str(move))
                    print("    S" + str(s_index_tmp))
                    print("Ended in S" + str(s_index))
                curr_board.pop()
                return s_index_tmp, move
            curr_board.pop()

        return -1, None
    else:
        if VERBOSE:
            print("---Black:---")
            print("Starting in S" + str(s_index))
        move = chess.Move.from_uci(STOCKFISH.get_best_move())
        curr_board.push(move)
        cur_fen = curr_board.fen()
        s_index = find_fen_in_sequence(cur_fen, s_n_sequence_short[:s_index])
        if VERBOSE:
            print("    Move: " + str(move))
            print("    S" + str(s_index))
            print("Ended in S" + str(s_index))
        curr_board.pop()
        return s_index, move

Bei der Berechnung der Züge für die KI wird die Funktion ``calculate_all_moves`` verwendet. Diese berechnet in einer Schleife alle Moves ausgehend von einer FEN, bis die KI gewonnen hat. Die Berechnung verwendet hierzu die FEN der Spielsituation (``fen``) und die Liste der gekürzten $S_n$ Mengen (``s_n_sequence_short``). Mithilfe von diesen Parametern wird eine Liste von Zügen bestimmt, in der die Partie zwischen Stockfish und der selbstgeschriebenen KI beendet wurde.

In [81]:
def calculate_all_moves(fen, s_n_sequence_short):
    moves = []

    s_index = find_fen_in_sequence(fen, s_n_sequence_short)

    board = chess.Board(fen)
    while s_index > 0:
        cur_fen = board.fen()
        s_index, next_move = find_next_move(cur_fen, s_index, s_n_sequence_short)
        board.push(next_move)
        moves.append(next_move)

    if s_index == -1:
        return None

    return moves

Nachdem eine Liste aller Züge für die KI bestimmt wurden, gilt dies gleichermaßen für die Berechnung von Stockfish umzusetzen. Hierfür erhält die Funktion ``stockfish_movelist`` eine ``fen`` mit der aktuellen Spielsituation. In der Funktion spielt Stockfish solange die bestmöglichen Züge, bist die Partie beendet worden ist. Diese Liste von Zügen wird am Ende der Funktion zurückgegeben.

In [82]:
def stockfish_movelist(fen):
    moves = []

    board = chess.Board(fen)

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move)

    return moves


### Hilfsfunktionen für den Vergleich

Der Vergleich zwischen der Stockfish Engine und der KI wird auf der Anzahl der Züge basieren, bis weiß das Spiel gewonnen hat. Hierfür werden zuerst drei Hilfsfunktionen definiert, die für die Durchführung des Vergleichs benötigt werden.

Bei der ersten Funktion handelt es sich um ``compare_move_list``. Diese erhält eine Liste von 3-Tupeln(``move_count_list``), die als ersten Wert die Anzahl der Züge der KI beinhaltet und als zweiten Wert die Anzahl der Züge, die Stockfish zum Beenden der Partie benötigt hat. Der dritte Wert stellt die Differenz zwischen den beiden Anzahlen dar.

Als Ergebnis liefert die Funktion drei Werte zurück, die ein erstes Abbild für die Perfomanz der KI darstellen. Der erste Wert ist die Anzahl der Spielsituationen, bei denen Stockfish und die KI gleich viele Züge benötigt haben (``equal``). Der zweite Wert ist die Anzahl der Spiele, bei denen die KI weniger Züge benötigt hat (``ki_better``) und der letzte Werte die Anzahl der Spiele, in der Stockish weniger Züge benötigt hat (``stockfish_better``).

In [83]:
def compare_move_lists(move_count_list):
    equal = 0
    ki_better = 0
    stockfish_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff == 0:
            equal += 1
        elif diff < 0:
            ki_better += 1
        else:
            stockfish_better += 1
    return equal, ki_better, stockfish_better

Diese Werte können nun weiter genutzt werden, indem neben der Anzahl der Ergebnisse auch der Grad des Unterschieds bestimmt wird. Hierfür wurde die Funktion ``get_average_difference`` definiert. Diese nutzt ebenfalls die Liste von 3-Tupeln (``move_count_list``) als Input. Die Funktion berechnet aus dieser Liste die durchschnittliche Prozentzahl an Züge, die die bessere Version weniger benötigt hat. Diese werden für sowohl die KI(``avg_ki_better``), als auch für Stockfish zurückgegeben(``avg_stock_better``).

In [84]:
def get_average_difference(move_count_list):
    percentual_ki = []
    percentual_stock = []
    avg_ki_better = 0
    avg_stock_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff < 0:
            percentual_ki.append(round(1 - (ki_move / stock_move), 4))
        elif diff > 0:
            percentual_stock.append(round(1 - (stock_move / ki_move), 4))
    if len(percentual_ki) != 0:
        avg_ki_better = sum(percentual_ki) / len(percentual_ki)
    if len(percentual_stock) != 0:
        avg_stock_better = sum(percentual_stock) / len(percentual_stock)
    return avg_ki_better, avg_stock_better

Die Ergebnisse des Vergleichs sollen abschließend in einer Datei gespeichert werden. Dies hat vor allem den Grund, dass man bei einer großen Anzahl von $S_n$ Mengen nicht die Konsole überfüllt und man zusätzlich die Ergebnisse lokal auf dem Rechner zur Verfügung hat. Aus diesem Grund wurde die Funktion ``write_result_to_file`` definiert. Diese berechnet zunächst mit der ``move_count_list`` die Ergebnisse des Vergleichs. Hierzu werden die zuvor definierten Funktionen ``compare_move_lists`` und ``get_average_difference`` aufgerufen. Außerdem werden die prozentualen Ergebnisse, welche Engine/KI das Problem besser gelöst hat, innerhalb dieser Funktion berechnet. Die Ergebnisse werden letztendlich in die Datei mit dem Namen ``filename`` in den Ordner Tests geschrieben. Zur Übersicht kann ein ``sequence_index`` mitgegeben werden, der eine Unterscheidung der Tests ermöglicht.

In [85]:
def write_result_to_file(filename, sequence_index, move_count_list):
    equal, ki_better, stockfish_better = compare_move_lists(move_count_list)
    avg_ki_better, avg_stock_better = get_average_difference(move_count_list)
    count = ki_better + stockfish_better + equal
    f = open("Tests/" + filename, "a+")
    f.write("S_" + str(sequence_index) + ":\n")
    f.write("Stockfish war zu " + str(round((stockfish_better / count) * 100, 2)) + "% besser.\n")
    f.write("Die KI war zu " + str(round((ki_better / count) * 100, 2)) + "% besser.\n")
    f.write("Stockfish und die KI haben zu " + str(
        round((equal / count) * 100, 2)) + "% die gleichen Ergebnisse erzielt.\n")
    f.write("Sofern die KI besser war, hat sie durchschnittlich " + str(
        round(avg_ki_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.write("Sofern Stockfish besser war, hat sie durchschnittlich " + str(
        round(avg_stock_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.close()

### Implementierung der Testszenarien
In diesem Abschnitt werden die zuvor definierten Testszenarien implementiert. Diese erhalten eine Liste der $S_n$ Mengen mit der verkürzten FEN Schreibweise (``s_n_sequence_short``) und oder die Liste der $S_n$ Mengen mit der vollständigen FEN Schreibweise (``s_n_sequence_fen``). Diese werden für die Berechnungen der Züge für die KI berechnet und müssen deshalb in mindestens einer Form für die Funktion vorliegen. Dabei gilt es für den Vergleich zu beachten, dass folgende Vorgehensweise verwendet wird.
- `Stockfish`: Für sowohl schwarz, als auch für weiß, werden die Züge mit der Funktion ``Stockfish().get_best_move()`` bestimmt.
- `KI`: Für weiß werden die Züge anhand der $S_n$ Mengen bestimmt, die schwarzen Züge werden mit der Funktion ``Stockfish().get_best_move`` gewählt.

Die Funktionen, die für den Vergleich mit Stockfish angewandt werden, bergen die Gefahr, dass durch Stockfish für schwarz nicht der beste Zug ausgewählt wird und dadurch das Ergebnis beeinflusst. Diese Gefahr erhöht sich vor allem dann, wenn sich die Anzahl der Züge bis zu einem Schachmatt erhöhen, da bei jedem Zug von schwarz die Gefahr besteht, dass nicht der beste Zug ausgewählt wird. Deshalb werden bei beiden Berechnungen die Züge für schwarz von Stockfish bestimmt, wobei das Risiko in geringem Maße bestehen bleibt.

Das erste Testszenario sieht die Überprüfung eines Spielszenarios vor (``fen``). Dies geschieht in der Funktion ``compare_fen_stockfish``. Hierfür werden nur die Anzahl der Züge von der KI und Stockfish miteinander verglichen. Das Ergebnis wird am Ende der Funktion in der Konsole ausgegeben.

In [86]:
def compare_fen_stockfish(fen, s_n_sequence_short):
    moves = calculate_all_moves(fen, s_n_sequence_short)
    stockfish_moves = stockfish_movelist(fen)

    if Moves is not None:
        print("AI needed " + str(len(moves)) + " moves to beat Stockfish as Black.")
    else:
        print("AI found no way to beat Black.")

    print("Stockfish needed " + str(len(stockfish_moves)) + " moves to win against itself.")

Das zweite Szenario sieht vor eine beliebige Anzahl (``count``) zufälliger Boards zu vergleichen. Anhand der ``count``-Variable werden aus den $S_n$ Mengen zufällige Boards ausgewählt und der Vergleich zwischen der KI und Stockfish zugeführt. Das Ergebnis wird in einer Datei hinterlegt.

In [95]:
import random


def test_random_boards(count, s_n_sequence_fen, s_n_sequence_short):
    count_s_n = len(s_n_sequence_fen)
    move_count_list = []
    for board_c in range(count):
        rand_sequence = random.randint(0, count_s_n - 1)
        rand_board = random.randint(0, len(s_n_sequence_fen[rand_sequence]) - 1)
        rand_fen = list(s_n_sequence_fen[rand_sequence])[rand_board]
        ki_moves = calculate_all_moves(rand_fen, s_n_sequence_short)
        stockfish_moves = stockfish_movelist(rand_fen)
        move_count = len(ki_moves)
        cmp_move_count = len(stockfish_moves)
        move_count_list.append(tuple((move_count, cmp_move_count, move_count - cmp_move_count)))
        clear_output()
        print("Analyzed " + str(board_c + 1) + "/" + str(count))

    filename = "Random_" + str(count) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                           "_") + ".txt"
    write_result_to_file(filename, "random", move_count_list)

In [96]:
test_random_boards(100, S_N_Sequence_fen, S_N_Sequence_fen_short)

Analyzed 100/100


Bei `compare_sequence_stockfish` wird für jedes Board einer $S_n$ Menge überprüft, ob Stockfish oder die zuvor definierten Mengen das Board effizienter lösen. In Form von ``sequence_index`` wird das $n$ der $S_n$ Menge übergeben. Zusätzlich kann ein Name für eine Datei angegeben werden, sodass mehrere Testergebnisse in einer Datei festgehalten werden können. Das Beschreiben der Datei erfolgt mit der Funktion ``write_result_to_file``.

In [89]:
def compare_sequence_stockfish(sequence_index, s_n_sequence_fen, s_n_sequence_short, g_filename=None):
    s_n = s_n_sequence_fen[sequence_index]
    move_count_list = []
    length = len(s_n)
    r = 0
    for board_fen in s_n:
        ai_moves = calculate_all_moves(board_fen, s_n_sequence_short)
        stockfish_moves = stockfish_movelist(board_fen)
        move_count = len(ai_moves)
        cmp_move_count = len(stockfish_moves)
        move_count_list.append(tuple((move_count, cmp_move_count, move_count - cmp_move_count)))
        clear_output()
        print("Comparing S_" + str(sequence_index) + ":")
        print("Compared " + str(r + 1) + "/" + str(length))
        r += 1

    if g_filename is None:
        filename = "S_" + str(sequence_index) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                                   "_") + ".txt"
        write_result_to_file(filename, sequence_index, move_count_list)
    else:
        write_result_to_file(g_filename, sequence_index, move_count_list)

Als Beispiel wird hier der Vergleich zwischen der KI und Stockfish für die Menge S_20 vorgenommen.

In [90]:
compare_sequence_stockfish(20, S_N_Sequence_fen, S_N_Sequence_fen_short)

Comparing S_20:
Compared 24/56


KeyboardInterrupt: 

Das letzte Testszenario sieht vor alle zuvor bestimmten Boards mit dem Lösungsweg von Stockfish zu vergleichen. Hierfür geht die Funktion `compare_all_sequences` alle $S_n$ Mengen durch und vergleicht diese jeweils mit der Lösung von Stockfish. Alle Ergebnisse werden in eine Datei geschrieben.

In [ ]:
def compare_all_sequences(s_n_sequence_fen, s_n_sequence_short):
    filename = "All_S_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":", "_") + ".txt"
    for i in range(len(s_n_sequence_fen)):
        print("Comparing S_" + str(i) + "...")
        compare_sequence_stockfish(i, s_n_sequence_fen, s_n_sequence_short, filename)

In [97]:
compare_all_sequences(S_N_Sequence_fen, S_N_Sequence_fen_short)

Comparing S_5:
Compared 3613/9064



KeyboardInterrupt



Für die visuelle Darstellung der Züge wurde die Funktion ``show_movelist`` definiert. Diese zeigt auf einem ``chess.Board`` eine mitgegebene Liste von Zügen. Für die Darstellung erhält die Funktion die ``fen``, die die Spielsituation zu Beginn der Partie darstellt. Außerdem wird eine Liste von Spielzügen, die dargestellt werden sollen, der Funktion mitgegeben (``moves``).

In [ ]:
def show_movelist(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)
